In [20]:
## ROBT 310 - Final Project - Virtual Keyboard
## Team: Daryn Kalym, Alibek Manabayev, Alexey Muryshkin
## Date: April 27, 2018

%matplotlib inline

## modules
import numpy as np
from math import sqrt
import cv2
import sys

## global variables
color_center = [255, 0, 0]
color_rad = 30
xMouse, yMouse = None, None
slctd_col = None
frame = None
magic_button = False
background = None

## functions

# Useless Function
def q(*x, **y):
    pass

def onBlueChange(value):
    global color_center
    color_center[0] = value
    
def onGreenChange(value):
    global color_center
    color_center[1] = value
    
def onRedChange(value):
    global color_center
    color_center[2] = value
    
def onRadiusChange(value):
    global color_rad
    color_rad = value
    
def onMagicButtonChange(value):
    global magic_button
    magic_button = bool(value)
    
def showColor(event, x, y, flags, param):
    global xMouse, yMouse, color_center, frame, slctd_col, magic_button
        
    if event == cv2.EVENT_LBUTTONDOWN:
        xMouse, yMouse = x, y
        slctd_col = tuple(frame[y - 30, x])
    elif event == cv2.EVENT_LBUTTONDBLCLK:
        xMouse, yMouse = x, y
        slctd_col = tuple(frame[y - 30, x])
        #if not magic_button:
        color_center = list( np.uint8( frame[y - 30, x] ) )
        
        
def processFrame(frame):
    global color_center, color_rad, xMouse, yMouse, slctd_col, magic_button
    
    font                   = cv2.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = (50,20)
    fontScale              = 0.8
    fontColor              = (255,255,255)
    lineType               = 2
    
    I2 = np.array(frame, copy = True)
    M, N, C = I2.shape

    values = np.repeat( np.mean(I2, axis = 2)[:,:,np.newaxis], 3, axis=2 )
    mask = np.reshape( np.repeat( (np.sum( (I2 - color_center)**2, axis = 2 ) > color_rad**2), 3 ), values.shape )
    I2[mask] = values[mask]

    black = np.zeros((30, I2.shape[1], 3), np.uint8)
    black[:] = (0, 0, 0)
    I2 = np.concatenate((black, I2), axis = 0)

    if xMouse and yMouse and slctd_col is not None:
        cv2.putText(I2,'x = ' + str(xMouse) + ', y = ' + str(yMouse - 30) + ', BGR(%d, %d, %d)' % slctd_col, 
            bottomLeftCornerOfText, 
            font, 
            fontScale,
            fontColor,
            lineType)
    
    return I2

def noiseFiltering(frame):
    grayImage = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    medFiltImage = cv2.medianBlur(grayImage, 25)
    
    return medFiltImage

def processFrame2(frame):
    global background
    threshold = 50
    
    frame = noiseFiltering(frame)
    mask = np.abs(frame - background) > threshold
    backSubImage = np.zeros(frame.shape)
    backSubImage[mask] = 255
    
    return backSubImage

def displayVideoRealTime(cap1, cap2):
    global color_center, color_rad, frame, magic_button, background
    
    cap = cap1
    ret, background = cap.read()
    background = noiseFiltering( np.flip( np.array(background, dtype=np.uint8), axis = 1 ) )
    
    window_name = "WebCam Video Streaming"
    cv2.namedWindow(window_name, flags=cv2.WINDOW_AUTOSIZE)
    cv2.moveWindow(window_name, 0, 0)
#     cv2.setMouseCallback(window_name, showColor)
    
#     trackbar_window = "Trackbar Window"
#     cv2.namedWindow(trackbar_window, flags=cv2.WINDOW_AUTOSIZE)
    
#     cv2.createTrackbar('Blue Color', trackbar_window, color_center[0], 255, onBlueChange)
#     cv2.createTrackbar('Green Color', trackbar_window, color_center[1], 255, onGreenChange)
#     cv2.createTrackbar('Red Color', trackbar_window, color_center[2], 255, onRedChange)
#     cv2.createTrackbar('Sphere Radius', trackbar_window, color_rad, int(sqrt(3 * 255**2)), onRadiusChange)
#     cv2.createTrackbar('Magic Button', trackbar_window, int(magic_button), 1, onMagicButtonChange)
    
#     cv2.moveWindow(trackbar_window, frame.shape[1], 0);
    
    while True:        
        if magic_button:
            cap = cap2
        else:
            cap = cap1
        
        ret, frame = cap.read()
        if not ret:
            break
            
        frame = np.flip( np.array(frame, dtype=np.uint8), axis = 1 )
#         I2 = processFrame(frame)
        I2 = processFrame2(frame)
        
        cv2.imshow(window_name, np.uint8( I2 ))
        
        key = cv2.waitKey(30)
        c = chr(key & 255)
        if c in ['q', 'Q', chr(27)]:
            break
    
    cap1.release()
    cap2.release()
    cv2.destroyAllWindows()
    

def getVideoCapture(idd):
    # initialization of video capture from video device {default: webcam}
#     if len(sys.argv) > 1 and sys.argv[1].isdigit():
#         cap = cv2.VideoCapture(int(sys.argv[1]))
#     else:
#         cap = cv2.VideoCapture(0)
            
#     cap_dev_id = input('Please enter the id of the opened video capturing device:\n')
    if isinstance(idd, int):
        cap = cv2.VideoCapture( idd )
    else:
        cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print('Cannot initialize video capture with id {}'.format(idd))
        #sys.exit(-1)
        
    return cap

    
## main program
def main():
    id1 = int( input('Please enter the id of the opened video capturing device #1:\n') )
    cap1 = getVideoCapture(id1)
#     id2 = int( input('Please enter the id of the opened video capturing device #2:\n') )
#     cap2 = getVideoCapture(id2)
    cap2 = None
    
    displayVideoRealTime(cap1, cap2)

    
if __name__ == '__main__':
    main()


Please enter the id of the opened video capturing device #1:
1
Please enter the id of the opened video capturing device #2:
0
